###  Libraries

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_kernels
from tslearn.clustering import KernelKMeans as TSKernelKMeans
from KernelKMeans import KernelKMeans
from GlobalKernelKMeans import GlobalKernelKMeans, GlobalKernelKMeansPP 
from Datasets.Datasets_Functions import *
from Rings import Rings
from Common_Modules.Evaluation import Evaluator
from Common_Modules.Graph import Graph
from Common_Modules.Visualization import Visualization
from Common_Modules.General_Functions import General_Functions

### Switches

In [2]:
plot_is_enabled = False
two_rings_is_enabled = False
three_rings_is_enabled = False
one_ring_with_two_gaussians_is_enabled = False
three_rings_paper_is_enabled = False
graph_is_enabled = True
real_dataset_is_enabled = False
nc_is_enabled = True
kMeans_is_enabled = False
kernelKMeans_is_enabled = False
kernelKMeansPP_is_enabled = True
globalKernelKMeans_is_enabled = True
globalKernelKMeansPP_is_enabled = True

### Global Variables

In [3]:
seed = 42
evaluator = Evaluator()
n_candidates = n_init = 50
kernels = ["cosine", "laplacian", "rbf"]
kernel = kernels[2]

### Visualization

In [4]:
visualization = Visualization()
colors = ['red', 'blue', 'green', 'orange', 'purple', 'yellow', 'black', 'cyan', 'gray', 'lime']

### Load Dataset

### 2Rings

In [5]:
if(two_rings_is_enabled):
    #centers_coordinates = [(-2, 2), (2, 2), (6, 2), (5, 6), (-1, 6), (2, 10), (5, -2), (2, -6), (-1, -2)] #Paper Rings
    #gamma = 1.0 #Paper Rings

    centers_coordinates = [(-2, 6), (2, 2), (6, 6), (-2, -2), (6, -2)] #Paper Rings Candidates
    gamma = 1.12 #Paper Rings Candidates
    rings = Rings()
    #gamma = 2.82
    X, y = rings.make_rings_pairs(centers_coordinates=centers_coordinates, n_samples=100, factor=0.2, noise=0.0)
    K = 2 * len(centers_coordinates)
    #gamma = General_Functions.calculate_gamma_scale(X)
    kernel_matrix = pairwise_kernels(X, X, metric='rbf', gamma=gamma)
    dataset_name = f"{K}Rings"
    data_dir_path = f"Results\\{dataset_name}\\{kernel}\\{n_init}\\{gamma}"
    General_Functions().create_directory(data_dir_path)
    visualization.plot(X, y, data_dir_path=f"{data_dir_path}Generated")

### 3Rings

In [6]:
if (three_rings_is_enabled):
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    rings = Rings()

    centers_coordinates = [(0, 0)]
    X, y = rings.make_concentric_rings(centers_coordinates=centers_coordinates, n_samples=150, radii=[1, 10, 25], noise=0)
    
    K = 3 * len(centers_coordinates)
    kernel_matrix = pairwise_kernels(X, X, metric='sigmoid')

### 1 Ring with 2 Gaussians Inside

In [7]:
if(one_ring_with_two_gaussians_is_enabled):
    rings = Rings()
    centers_coordinates = [(0, 0), (15,0), (30,0)]
    X, y = rings.make_multiple_rings_with_gaussians(centers_coordinates, n_samples=50, radius=4.25, noise=0.0, gaussian_samples=50)
    gamma = 0.08
    K = 3 * len(centers_coordinates)
    kernel_matrix = pairwise_kernels(X, X, metric=kernel, gamma=gamma)
    dataset_name = f"{int(K/3)}Rings_{int((2*K)/3)}Gaussians"
    data_dir_path = f"Results\\{dataset_name}\\{kernel}\\{n_init}\\{gamma}\\"
    General_Functions().create_directory(data_dir_path)
    visualization.plot(X, y, data_dir_path=f"{data_dir_path}Generated")

### 3Rings Paper Tzortzis

In [8]:
if (three_rings_paper_is_enabled):
    rings = Rings(colors=colors)
    K = 3
    X, y, kernel_matrix = rings.global_kernel_k_means_three_rings()
    rings.plot(X,y)

### Graphs

In [ ]:
if(graph_is_enabled):
    K = 10
    graph = Graph(n_communities=K, n_nodes_per_community=10, p_intra=1, p_inter=0.01, colors=colors)
    dataset_name = "email-Eu-core"
    graph.create_G_from_file(f"C:\\Users\\maria\\Documents\\GitHub\\global-kernel-k-means-pp\\Graphs\\{dataset_name}.txt")
    graph.create_adj_matrix()
    graph.create_degrees_matrix()
    if(nc_is_enabled):
        kernel_matrix = graph.create_kernel_matrix_from_adj_matrix(D = graph.degree_matrix)
    else:
        kernel_matrix = graph.create_kernel_matrix_from_adj_matrix()
    data_dir_path = f"Results\\{dataset_name}\\{n_init}\\"
    General_Functions().create_directory(data_dir_path)

### Real Dataset

In [10]:
if (real_dataset_is_enabled):
    dataset_names = ["breast_cancer", "dermatology", "ecoli", "iris", "olivetti_faces", "pendigits", "waveform_v1", "wine"]
    dataset_name = dataset_names[5] 
    module_name = "Datasets.Datasets"
    batch_size = 16
    dataloader, input_dim, X, y = function_get_dataset(dataset_name, module_name, batch_size)
    print('Date Number is:', len(y))
    print('Data_Shape is:', input_dim)
    K = len(np.unique(y))
    gamma = General_Functions.calculate_gamma_scale(X)
    kernel_matrix = pairwise_kernels(X, X, metric=kernel, gamma=gamma)
    data_dir_path = f"Results\\{dataset_name}\\{kernel}\\{n_init}\\{gamma}\\"
    General_Functions().create_directory(data_dir_path)
    K = 50

### kMeans

In [11]:
if(kMeans_is_enabled):
    kmeans = KMeans(n_clusters=K).fit(X)
    acc, pur, nmi, ari = evaluator.evaluate_model(y, kmeans.labels_)
    evaluator.print_evaluation()
    if(plot_is_enabled):
        visualization.plot_tsne(X, y, data_dir_path="Figures\\"+dataset_name)

### Kernel kMeans

In [12]:
if(kernelKMeans_is_enabled):
    kernel_kmeans = TSKernelKMeans(n_clusters=K, n_init=10, kernel='precomputed').fit(kernel_matrix)
    acc, pur, nmi, ari = evaluator.evaluate_model(y, kernel_kmeans.labels_)
    evaluator.print_evaluation()
    if(plot_is_enabled):
        visualization.plot_tsne(X, y, data_dir_path="Figures\\"+dataset_name)

### Kernel kMeans++

In [21]:
if(kernelKMeansPP_is_enabled):
    initialization_methods = ["forgy", "random", "k-means++"]
    n_iters_ = {}
    execution_times_ = {}
    inertias_ = {}
    init = initialization_methods[1]
    prev_K = 100
    K = 100
    while (K != prev_K + 1 ):
        kernelKMeans = KernelKMeans(n_clusters=K, kernel_matrix=kernel_matrix, n_init=n_init, init=init, verbose=0)
        if(nc_is_enabled):
            kernelKMeans.fit(sample_weights = graph.weights)
        else:
            kernelKMeans.fit()    
        #acc, pur, nmi, ari = evaluator.evaluate_model(y, kernelKMeans.labels_)
        #evaluator.print_evaluation()
        if(plot_is_enabled):
            if graph_is_enabled:
                graph.plot_clusters(kernelKMeans.labels_)
            else:
                visualization.plot(X, kernelKMeans.labels_, data_dir_path=data_dir_path + "KernelKMeans_" + init)                
        
        n_iters_[K] = sum(kernelKMeans.n_iters_.values())
        execution_times_[K] = sum(kernelKMeans.execution_times_.values())
        inertias_[K] = kernelKMeans.inertia_
        new_row = { "K": K, "MSE": inertias_[K], "ITERATIONS": n_iters_[K], "EXECUTION TIME": execution_times_[K]}
        General_Functions.append_to_csv(f"{data_dir_path}KernelKMeans_Dataset_{dataset_name}_Init_{init}_Kernel_{kernel}_K_{prev_K}_Inits_{n_init}_Results.csv", new_row)
        K += 1
    K = prev_K   

In [22]:
print(new_row)

{'K': 100, 'MSE': np.float64(949.0926618099677), 'ITERATIONS': 62, 'EXECUTION TIME': 3.607191324234009}


### Global Kernel kMeans

In [ ]:
if(globalKernelKMeans_is_enabled):
    globalKernelKMeans = GlobalKernelKMeans(n_clusters=K, kernel_matrix=kernel_matrix, data_dir_path=f"{data_dir_path}GlobalKernelKMeans_Dataset_{dataset_name}_Kernel_{kernel}_K_{K}_Inits_{n_init}_Results.csv", verbose=1)
    if(nc_is_enabled):
        globalKernelKMeans.fit(sample_weights = graph.weights)
    else:
        globalKernelKMeans.fit()    
    #acc, pur, nmi, ari = evaluator.evaluate_model(y, globalKernelKMeans.labels_[K]) evaluator.print_evaluation()
    if(plot_is_enabled):
        if graph_is_enabled:
            graph.plot_clusters(globalKernelKMeans.labels_[K])
        else:
            visualization.plot(X, globalKernelKMeans.labels_[K], data_dir_path=f"{data_dir_path}GlobalKernelKMeans")     

In [16]:
if(globalKernelKMeansPP_is_enabled):
    #best_acc = 0
    #best_gamma = 0
    #while(best_acc != 1.0):
    sampling_methods = ["sequential", "batch"]
    sampling = sampling_methods[1]
    globalKernelKMeansPP = GlobalKernelKMeansPP(n_clusters=K, kernel_matrix=kernel_matrix, n_candidates=n_candidates, sampling=sampling, data_dir_path=f"{data_dir_path}GlobalKernelKMeans_Dataset_{dataset_name}_Sampling_{sampling}_Kernel_{kernel}_K_{K}_Inits_{n_init}_Results.csv", verbose=1)
    if(nc_is_enabled):
        globalKernelKMeansPP.fit(sample_weights = graph.weights)
    else:
        globalKernelKMeansPP.fit()    

    #acc, pur, nmi, ari = evaluator.evaluate_model(y, globalKernelKMeansPP.labels_[K])
    """if(acc> best_acc):
        best_acc = acc
        best_gamma = gamma
        print(f"{acc}, {gamma}")
    gamma += 0.01"""
    if(plot_is_enabled):
        if graph_is_enabled:
            graph.plot_clusters(globalKernelKMeansPP.labels_[K])
        else:
            visualization.plot(X, globalKernelKMeansPP.labels_[K], data_dir_path=f"{data_dir_path}GlobalKernelKMeans_{sampling}_")

Solved 34-means MSE: 1000.3792964051237 in 1.6948542594909668s
Solving 35-means
Solved 35-means MSE: 998.8793269926873 in 1.7174885272979736s
Solving 36-means
Solved 36-means MSE: 997.5460548298186 in 1.71061110496521s
Solving 37-means
Solved 37-means MSE: 996.2127826781687 in 1.6911933422088623s
Solving 38-means
Solved 38-means MSE: 994.8795105377407 in 1.6751675605773926s
Solving 39-means
Solved 39-means MSE: 993.3795411486833 in 1.6768145561218262s
Solving 40-means
Solved 40-means MSE: 991.8795717633695 in 1.6426115036010742s
Solving 41-means
Solved 41-means MSE: 990.5462996472683 in 1.6290724277496338s
Solving 42-means
Solved 42-means MSE: 989.2130275423988 in 1.6506457328796387s
Solving 43-means
Solved 43-means MSE: 987.8797554487644 in 1.6803910732269287s
Solving 44-means
Solved 44-means MSE: 986.546483366368 in 1.6411969661712646s
Solving 45-means
Solved 45-means MSE: 985.2965752479448 in 1.6772410869598389s
Solving 46-means
Solved 46-means MSE: 984.0466671520087 in 1.7137126922